In [ ]:
%env GOOGLE_CLOUD_PROJECT = physionet-data-403309 #Change to your own project

env: GOOGLE_CLOUD_PROJECT=physionet-data-403309 #Change to your own project


In [ ]:
from google.colab import files
from google.cloud import bigquery
from google.colab import auth
auth.authenticate_user()
print('Authenticated')
%reload_ext google.colab.data_table
%load_ext google.colab.data_table

In [33]:
def get_vars_df(sheet_id, gid):
  return pd.read_csv(f'https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv&gid={gid}')

##Suspected Sepsis (Cohort A)

In [ ]:
%%bigquery dataA --project physionet-data-403309

WITH source_data AS (
  SELECT
    hadm_id

    , MIN(suspected_infection_time) AS min_charttime
  FROM `physionet-data.mimiciv_derived.suspicion_of_infection`
  GROUP BY hadm_id
),

sofa as (
  SELECT hadm_id, icu.stay_id, starttime, sofa_24hours
  FROM `physionet-data.mimiciv_derived.sofa` as sof
  left join `physionet-data.mimiciv_icu.icustays` as icu on sof.stay_id = icu.stay_id
),

FilteredSOFA AS (
  SELECT
    s.hadm_id,
    s.sofa_24hours as sofa_score,
    ROW_NUMBER() OVER (PARTITION BY s.hadm_id ORDER BY s.starttime ASC) AS rn
  FROM
    sofa s
  JOIN
    source_data sd ON s.hadm_id = sd.hadm_id
    AND s.starttime BETWEEN TIMESTAMP_ADD(sd.min_charttime, INTERVAL -24 HOUR) AND TIMESTAMP_ADD(sd.min_charttime, INTERVAL 24 HOUR)
),

final_sofa as(SELECT
  sd.hadm_id,
  sd.min_charttime,
  fs.sofa_score AS sofa_score
FROM
  source_data sd
LEFT JOIN
  FilteredSOFA fs ON sd.hadm_id = fs.hadm_id AND fs.rn = 1)
, eligibility as (SELECT
  source_data.hadm_id
  , source_data.min_charttime
  , adm.admittime
  , DATE_DIFF(source_data.min_charttime, admittime, hour) as time_to_culture
  , DATE_DIFF(adm.dischtime, source_data.min_charttime, hour) as time_to_discharge
  ,(SELECT
    COUNT(*)
    FROM `physionet-data.mimiciv_icu.icustays` icu
    WHERE icu.hadm_id = source_data.hadm_id AND icu.intime > source_data.min_charttime
  ) AS icu_stay_count_after_culture
  ,(SELECT
    COUNT(*)
    FROM `physionet-data.mimiciv_icu.icustays` icu
    WHERE icu.hadm_id = source_data.hadm_id AND icu.intime < source_data.min_charttime
  ) AS icu_stay_count_before_culture
  ,(
      SELECT MIN(DATE_DIFF(icu.intime, source_data.min_charttime, HOUR))
      FROM `physionet-data.mimiciv_icu.icustays` icu
      WHERE icu.hadm_id = source_data.hadm_id AND icu.intime > source_data.min_charttime
    ) AS time_to_first_icu_after_culture
  ,(
    SELECT EXISTS (
      SELECT 1
      FROM `physionet-data.mimiciv_icu.icustays` icu
      WHERE icu.hadm_id = source_data.hadm_id
        AND source_data.min_charttime BETWEEN icu.intime AND icu.outtime
    )
  ) AS culture_during_icu

FROM
  source_data
LEFT JOIN `physionet-data.mimiciv_hosp.admissions` adm ON
  source_data.hadm_id = adm.hadm_id

WHERE
  DATE_DIFF(min_charttime, admittime, HOUR) >= 48
  and DATE_DIFF(adm.dischtime, min_charttime, HOUR) >= 4)


SELECT
    eg.hadm_id,
    eg.min_charttime,
    eg.admittime,
    time_to_culture,
    time_to_discharge,
    icu_stay_count_after_culture,
    icu_stay_count_before_culture,
    time_to_first_icu_after_culture
    , culture_during_icu
    , sof.sofa_score
    , ad.discharge_location
    , ad.hospital_expire_flag AS in_hospital_mortality
-- Demographic
    , age.age
    , ad.race as ethnicity
    , CASE WHEN ad.language = "ENGLISH" THEN 1 ELSE 0 END AS language_english
    , pat.gender
    , ad.insurance AS insurance
--Charlson
    ,"Charlson" AS comorbidity_score_name
    , charlson.charlson_comorbidity_index AS comorbidity_score_value
    , charlson.age_score
    , charlson.myocardial_infarct
    , charlson.congestive_heart_failure
    , charlson.peripheral_vascular_disease
    , charlson.cerebrovascular_disease
    , charlson.dementia
    , charlson.chronic_pulmonary_disease
    , charlson.rheumatic_disease
    , charlson.peptic_ulcer_disease
    , charlson.mild_liver_disease
    , charlson.diabetes_without_cc
    , charlson.diabetes_with_cc
    , charlson.paraplegia
    , charlson.renal_disease
    , charlson.malignant_cancer
    , charlson.severe_liver_disease
    , charlson.metastatic_solid_tumor
    , charlson.aids

from eligibility as eg

-- Charlson
LEFT JOIN physionet-data.mimiciv_derived.charlson
AS charlson
ON eg.hadm_id = charlson.hadm_id
-- Admissions
LEFT JOIN physionet-data.mimiciv_hosp.admissions
AS ad
ON eg.hadm_id = ad.hadm_id
-- Patients
LEFT JOIN physionet-data.mimiciv_derived.age
AS age
ON eg.hadm_id = age.hadm_id
-- Patients
LEFT JOIN physionet-data.mimiciv_hosp.patients
AS pat
ON ad.subject_id = pat.subject_id

LEFT JOIN final_sofa
as sof
on eg.hadm_id = sof.hadm_id

In [36]:
len(dataA)

12041

In [37]:
project_id = "physionet-data-403309"
dataset_id = "DukeDatathon"

final_tables = {"data_A": dataA}

for t_name, t_df in final_tables.items():
  t_df = t_df.astype(str)
  table_id = create_table_id(project_id, dataset_id, t_name)
  t_df.to_gbq(table_id, project_id=project_id, if_exists='replace')

100%|██████████| 1/1 [00:00<00:00, 1640.96it/s]


##Culture data (cohort B)

In [ ]:
%%bigquery dataB --project physionet-data-403309

with source_data AS (
  SELECT
    hadm_id
    , MIN(charttime) AS min_charttime
  FROM `physionet-data.mimiciv_hosp.microbiologyevents`
  WHERE
    org_name IS NOT NULL
    AND org_name != ''
    AND org_itemid != 90856
    AND hadm_id IS NOT NULL
    AND spec_type_desc IN( 'BLOOD CULTURE', 'SEROLOGY/BLOOD')
  GROUP BY hadm_id
),

sofa as (
  SELECT hadm_id, icu.stay_id, starttime, sofa_24hours
  FROM `physionet-data.mimiciv_derived.sofa` as sof
  left join `physionet-data.mimiciv_icu.icustays` as icu on sof.stay_id = icu.stay_id
),

FilteredSOFA AS (
  SELECT
    s.hadm_id,
    s.sofa_24hours as sofa_score,
    ROW_NUMBER() OVER (PARTITION BY s.hadm_id ORDER BY s.starttime ASC) AS rn
  FROM
    sofa s
  JOIN
    source_data sd ON s.hadm_id = sd.hadm_id
    AND s.starttime BETWEEN TIMESTAMP_ADD(sd.min_charttime, INTERVAL -24 HOUR) AND TIMESTAMP_ADD(sd.min_charttime, INTERVAL 24 HOUR)
),

final_sofa as(SELECT
  sd.hadm_id,
  sd.min_charttime,
  fs.sofa_score AS sofa_score
FROM
  source_data sd
LEFT JOIN
  FilteredSOFA fs ON sd.hadm_id = fs.hadm_id AND fs.rn = 1),
  eligibility as (SELECT
  source_data.hadm_id
  , source_data.min_charttime
  , adm.admittime
  , DATE_DIFF(source_data.min_charttime, admittime, hour) as time_to_culture
  , DATE_DIFF(adm.dischtime, source_data.min_charttime, hour) as time_to_discharge
  ,(SELECT
    COUNT(*)
    FROM `physionet-data.mimiciv_icu.icustays` icu
    WHERE icu.hadm_id = source_data.hadm_id AND icu.intime > source_data.min_charttime
  ) AS icu_stay_count_after_culture
  ,(SELECT
    COUNT(*)
    FROM `physionet-data.mimiciv_icu.icustays` icu
    WHERE icu.hadm_id = source_data.hadm_id AND icu.intime < source_data.min_charttime
  ) AS icu_stay_count_before_culture
  ,(
      SELECT MIN(DATE_DIFF(icu.intime, source_data.min_charttime, HOUR))
      FROM `physionet-data.mimiciv_icu.icustays` icu
      WHERE icu.hadm_id = source_data.hadm_id AND icu.intime > source_data.min_charttime
    ) AS time_to_first_icu_after_culture
  ,(
    SELECT EXISTS (
      SELECT 1
      FROM `physionet-data.mimiciv_icu.icustays` icu
      WHERE icu.hadm_id = source_data.hadm_id
        AND source_data.min_charttime BETWEEN icu.intime AND icu.outtime
    )
  ) AS culture_during_icu

FROM
  source_data
LEFT JOIN `physionet-data.mimiciv_hosp.admissions` adm ON
  source_data.hadm_id = adm.hadm_id

WHERE
  DATE_DIFF(min_charttime, admittime, HOUR) >= 48
  and DATE_DIFF(adm.dischtime, min_charttime, HOUR) >= 4)

select
eg.hadm_id,
eg.min_charttime,
eg.admittime,
time_to_culture,
time_to_discharge,
icu_stay_count_after_culture,
icu_stay_count_before_culture,
time_to_first_icu_after_culture
, culture_during_icu
, sof.sofa_score
, ad.discharge_location
, ad.hospital_expire_flag AS in_hospital_mortality
-- Demographic
, age.age
, ad.race as ethnicity
, CASE WHEN ad.language = "ENGLISH" THEN 1 ELSE 0 END AS language_english
, pat.gender
, ad.insurance AS insurance
--Charlson
,"Charlson" AS comorbidity_score_name
, charlson.charlson_comorbidity_index AS comorbidity_score_value
, charlson.age_score
, charlson.myocardial_infarct
, charlson.congestive_heart_failure
, charlson.peripheral_vascular_disease
, charlson.cerebrovascular_disease
, charlson.dementia
, charlson.chronic_pulmonary_disease
, charlson.rheumatic_disease
, charlson.peptic_ulcer_disease
, charlson.mild_liver_disease
, charlson.diabetes_without_cc
, charlson.diabetes_with_cc
, charlson.paraplegia
, charlson.renal_disease
, charlson.malignant_cancer
, charlson.severe_liver_disease
, charlson.metastatic_solid_tumor
, charlson.aids

from eligibility as eg

-- Charlson
LEFT JOIN physionet-data.mimiciv_derived.charlson
AS charlson
ON eg.hadm_id = charlson.hadm_id
-- Admissions
LEFT JOIN physionet-data.mimiciv_hosp.admissions
AS ad
ON eg.hadm_id = ad.hadm_id
-- Age
LEFT JOIN physionet-data.mimiciv_derived.age
AS age
ON eg.hadm_id = age.hadm_id
-- Patient
LEFT JOIN physionet-data.mimiciv_hosp.patients
AS pat
ON ad.subject_id = pat.subject_id
--SOFA
LEFT JOIN final_sofa
as sof
on eg.hadm_id = sof.hadm_id

In [39]:
len(dataB)

2822

Upload data

In [40]:
project_id = "physionet-data-403309"
dataset_id = "DukeDatathon"

final_tables = {"data_B": dataB}

for t_name, t_df in final_tables.items():
  t_df = t_df.astype(str)
  table_id = create_table_id(project_id, dataset_id, t_name)
  t_df.to_gbq(table_id, project_id=project_id, if_exists='replace')

100%|██████████| 1/1 [00:00<00:00, 7958.83it/s]
